In [1]:
import VoltageGeneration

In [2]:
import os
import os.path
import time

In [3]:
%reload_ext autoreload
%matplotlib tk
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize as sciopt
import pandas as pd

colors = ["#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059", \
          "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", \
          "#8FB0FF", "#997D87", "#5A0007", "#809693", "#FFAA92", "#1B4400", \
          "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80", "#61615A", "#BA0900", \
          "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100", \
          "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", \
          "#013349", "#00846F", "#372101", "#FFB500", "#C2FFED", "#A079BF", \
          "#CC0744", "#C0B9B2", "#C2FF99", "#001E09", "#00489C", "#6F0062"]

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
amu = 1.66e-27
elem_charge = 1.602e-19
m = 40 * amu

kHz = 1e3
MHz = 1e6

In [5]:
# Ordering for the SLT configuration
_electrode_ordering = [
    "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
    "S24", "S25",
    "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
]
# converts grid file number to electrode

def num_to_electrode_name(num):
    return _electrode_ordering[num-1]

artiq_electrode_ordering = [12, 13, 14, 15, 16, 17, 18, 19, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [5]:
# Ordering for the maxbeta configuration
_electrode_ordering = [
    "S1", "S2", "S3", "S4", "S8", "S9", "S10", "S11", "S12",
    "S13", "S14", "S19", "S20", "S21",
    "S22", "S23", "S24", "S25",
    "N1", "N2", "N3", "N4", "N8", "N9", "N10", "N11", "N12",
    "N13", "N14", "N19", "N20", "N21",
]
# converts grid file number to electrode

def num_to_electrode_name(num):
    grid_num = artiq_electrode_ordering.index(num)
    return _electrode_ordering[grid_num]

artiq_electrode_ordering = [27, 25, 23, 21, 19, 17, 15, 13, 11,
                            9, 7, 5, 3, 1,
                            32, 31, 30, 29,
                            2, 4, 6, 8, 10, 12, 14, 16, 18,
                            20, 22, 24, 26, 28] ## maps electrode name : grid file number

In [6]:
#COMSOL grid files
path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = "MaxBeta_Production_Potential"
name = "AngleTrap_Skinny_Potential_reduced"

electrode_grouping = [[x] for x in artiq_electrode_ordering]
d_cons = [('x',0), ('y',0), ('z',0)]

start_time = time.time()

vg_comsol = VoltageGeneration.VoltageGeneration(
    path,
    name,
    electrode_grouping,
    fit_ranges=[5e-6, 10e-6, 5e-6],
    rf_electrodes=[21], ## 33 for MaxBeta
    m=m,
    rf_omega=2.*np.pi*45.247E6,
    order=2,
    f_cons=d_cons,
    v_rf=[50]
)

print("Imported grid files in {}".format(time.time() - start_time))

Importing electrode 18
Importing electrode 19
Importing electrode 21
Importing electrode 20
Importing electrode 1
Importing electrode 2
Importing electrode 3
Importing electrode 7
Importing electrode 6
Importing electrode 4
Importing electrode 5
Importing electrode 8
Importing electrode 9
Importing electrode 12
Importing electrode 13
Importing electrode 11
Importing electrode 10
Importing electrode 14
Importing electrode 15
Importing electrode 17
Importing electrode 16
Potentials imported
Imported grid files in 30.157999992370605


In [7]:
def rotate_coeffs(theta, xx, xz, zz):
    """
    Find the coefficients $C'_{xx}, C'_{xz}, C'_{zz}$ of the potential
    after a rotation in the zx-plane by `theta` given the unprimed values,
    as described above.
    
    `theta` specified in radians.
    """
    sin = np.sin
    cos = np.cos
    
    xxnew = xx*cos(theta)**2 + zz*sin(theta)**2 + 0.5*xz*sin(2*theta)
    xznew = xz*cos(2*theta)  + (zz-xx)*sin(2*theta)
    zznew = zz*cos(theta)**2 + xx*sin(theta)**2 - 0.5*xz*sin(2*theta)
    
    return [xxnew, xznew, zznew]

In [8]:
def dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    # expansion coefficients of the pseudopotential near the trapping point `R`
    # THIS FUNCTION ASSUMES ONE RF ELECTRODE!
    rf_coeffs = vg.printCoefficients(R, rfEls, ('zz', 'xx', 'xz', 'yy', 'xy', 'yz'), printing=False)[0]
    [rfzz, rfxx, rfxz, rfyy, rfxy, rfyz] = rf_coeffs[:,0]
    
    def tiltError(theta_tilt, debug=False):
        """
        Error in the tilt angle of the full potential, as a function of the dc tilt.
        """
        
        zz = delta*yy
        xx = -(1+delta)*yy # delta ensures Laplace (div phi = 0) is satisfied, but adjusts the principal axes of the xz ellipse
        xz = 0
        
        # rotate!
        [xxnew, xznew, zznew] = rotate_coeffs(theta_tilt, xx, xz, zz)

        if debug:
            print (np.degrees(theta_tilt))
            print (np.degrees(np.arctan((xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))))/2)
        
        return (
            np.tan(2*theta_targ) - (xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))
        )**2
    
    if debug:
        print (np.degrees(theta_targ))

        fig, ax = plt.subplots()
        angle_space = np.linspace(0, np.pi/2, 91)
        errors = [tiltError(theta) for theta in angle_space]
        ax.plot(np.degrees(angle_space), errors)
        ax.set_ylim(0,1)
    
    return sciopt.minimize_scalar(tiltError).x

def tiltWithPseudo(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    th = dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=delta, debug=debug)
    
    if debug:
        print ("Tilting dc to {:.2f} deg".format(np.degrees(th)))
    
    zz = delta*yy
    xx = -(1+delta)*yy
    xz = 0
    return rotate_coeffs(th, xx, xz, zz)

**Getting Trapping Voltages**

In [14]:
####
voltage_gen = vg_comsol

sec_freq = 1050*kHz
omega = 2 * np.pi * sec_freq

trap_center = [0.0, 0, 50.0]
R = [trap_center]

tilt_angle_deg = 15 #in degrees
tilt_angle = np.radians(tilt_angle_deg)

yycons = 0.5 * m * omega**2 * (1e-6)**2 / elem_charge
print(yycons)
[xxt, xzt, zzt] =  tiltWithPseudo(voltage_gen, R, tilt_angle, yycons, voltage_gen.rf_electrodes, debug=False)
cons = [('xx', xxt), ('yy', yycons), ('zz', zzt), ('xz', xzt), ('yz', 0), ('xy', 0)]

print("target alpha = %.2E"%(yycons/(1e-6)**2))

# constrained_voltages = [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0),
#                         (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0),
#                         (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0),
#                         (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0),
#                         (30, 0.0), (31, 0.0),
#                        ]

# constrained_voltages = [(i, 0.0) for i, v in enumerate(artiq_electrode_ordering) if v not in [1,2,3,4,5,6,7,29,30,31,32]]

constrained_voltages = []

nom_voltages = voltage_gen.findControlVoltages(
    R,
    cons=[cons],
    tol=1e-18,
    fixed_voltages=constrained_voltages,
    bnds =(-9.9, 9.9),
)

el_config = []
for [num, voltage] in voltage_gen.ungroup_configuration(nom_voltages):
    el_config.append([[num], voltage])

print(el_config)

_ = voltage_gen.compute_total_potential_axes(R, el_config, printing=True)

9.020153056006835e-06
target alpha = 9.02E+06
Optimization failed after 2001 iterations.
Target, Realized Coeffs:
 [[ 0.00000000e+00 -8.99835346e-07]
 [ 0.00000000e+00 -9.53582622e-07]
 [ 0.00000000e+00 -4.95515263e-07]
 [-3.25070557e-05 -3.27090822e-05]
 [ 0.00000000e+00  1.01988153e-07]
 [ 2.91799850e-05  2.91455382e-05]
 [ 9.02015306e-06  9.35143507e-06]
 [ 0.00000000e+00 -1.82910499e-08]
 [ 2.34869026e-05  2.33481914e-05]]
Final cost value:  1.2755106404642187e-05
Number of iterations:  2001 

[[[12], 1.8997819344122333], [[13], 1.0777112523619026], [[14], 2.091697810949457], [[15], 0.8138836143258498], [[16], -7.386575974773923], [[17], 0.8696320114163281], [[18], 2.1077928755434563], [[19], 1.079724075950516], [[20], 1.7576595766740104], [[1], 1.9744426737566854], [[2], 1.151782589716637], [[3], 2.27834023965346], [[4], 1.369268889055969], [[5], -6.41039584337887], [[6], 1.4155193684645158], [[7], 2.289030883402544], [[8], 1.149851149941443], [[9], 1.8262381157958465], [[10], -0.

**Export voltage set to a simple npy file**

In [15]:
####
voltage_set_to_export = nom_voltages
file_path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
file_name = os.path.join(file_path, "NominalVolts_1050kHz_0um.npy")

print(file_name)

el_config = []
for [num, voltage] in voltage_gen.ungroup_configuration(nom_voltages):
    el_config.append([[num], voltage])
    
print(el_config)

voltages_dict = {num_to_electrode_name(num): v for [[num], v] in el_config}

print(voltages_dict)

np.save(file_name, voltages_dict)

/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/NominalVolts_1050kHz_0um.npy
[[[12], 1.8997819344122333], [[13], 1.0777112523619026], [[14], 2.091697810949457], [[15], 0.8138836143258498], [[16], -7.386575974773923], [[17], 0.8696320114163281], [[18], 2.1077928755434563], [[19], 1.079724075950516], [[20], 1.7576595766740104], [[1], 1.9744426737566854], [[2], 1.151782589716637], [[3], 2.27834023965346], [[4], 1.369268889055969], [[5], -6.41039584337887], [[6], 1.4155193684645158], [[7], 2.289030883402544], [[8], 1.149851149941443], [[9], 1.8262381157958465], [[10], -0.21324968108903014], [[11], 0.4210816168618554]]
{'N6': 1.8997819344122333, 'N7': 1.0777112523619026, 'N8': 2.091697810949457, 'N9': 0.8138836143258498, 'N10': -7.386575974773923, 'N11': 0.8696320114163281, 'N12': 2.1077928755434563, 'N13': 1.079724075950516, 'N14': 1.7576595766740104, 'S6': 1.9744426737566854, 'S7': 1.151782589716637, 'S8': 2.27834023965346, 'S9': 1.369268889055969, 'S10': -6.41039584337887, 'S11

**Load simple npy file and print fit results**

In [ ]:
#myvolts = np.load('N:\\Individual Folders\\Stuart\\xfer\\NominalVolts_20200310_MZTWINWAC_1p20_0um.npy').item()
myvolts = np.load('N:\\Individual Folders\\Stuart\\xfer\\NominalVolts_20200527_NOHOLENITO_0p60_0um.npy').item()

voltages = []
for electrode_name in _electrode_ordering:
    voltages.append(myvolts[electrode_name])

print(voltages)
    
el_config = []
for index, voltage in enumerate(voltages):
    el_config.append([[index+1], voltage])

print(el_config)
    
expansion_point = [[0., 0.0, 50.]]

#vg.compute_total_potential_axes(expansion_point, el_config, printing=True)
voltage_gen.compute_total_potential_axes(expansion_point, el_config, printing=True)

## Simplified version of nominal and shim exports

In [26]:
voltage_gen = vg_comsol
output_path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filename_spec = '20201119_MAXBETA_5deg_1000kHz_-373um'
axial_freq = 1000*kHz #800kHz
tilt = 5
trap_pos = [0, -186.5, 50]
constrained_voltages = [(i, 8) for i, v in enumerate(artiq_electrode_ordering) if v not in [1,16,17,18,19,20,23,24,25,26,27,29,30,31,32]]
constrained_voltages = constrained_voltages + [(artiq_electrode_ordering.index(29), -5.597347664697586), (artiq_electrode_ordering.index(30), 5.2902534215513), (artiq_electrode_ordering.index(31), -5.617763846316864), (artiq_electrode_ordering.index(32), 4.87897587086919)]
constrained_voltages = constrained_voltages + [(artiq_electrode_ordering.index(23), -1.063478070573189),(artiq_electrode_ordering.index(24), 0.11024014905738937),(artiq_electrode_ordering.index(25), 0.48728710866056835),(artiq_electrode_ordering.index(26), -0.09088604221675678),(artiq_electrode_ordering.index(27), 0.6757519347558454)]
# [[16], -3.1345600112211294], [[17], 0.7957012684322433], [[18], 0.1796437443686206], [[19], 0.7866260497899049], [[20], -3.0611287716426503]
constrained_voltages = constrained_voltages + [(artiq_electrode_ordering.index(16), -3.1345600112211294),(artiq_electrode_ordering.index(17), 0.7957012684322433),(artiq_electrode_ordering.index(18), 0.1796437443686206),(artiq_electrode_ordering.index(19), 0.7866260497899049),(artiq_electrode_ordering.index(20), -3.0611287716426503)]
print(constrained_voltages)

###

omega = 2 * np.pi * axial_freq
convexity = 1 if axial_freq > 0 else -1  # allow anti-confining potentials
tilt_angle = np.radians(tilt)
R = [trap_pos]
# R = [[0, 186.5, 50], [0, 559.5, 50]]

yycons = 0.5 * m * convexity * omega**2 * (1e-6)**2 / elem_charge
## try with getting tilt coefficients with different delta values
[xxt, xzt, zzt] =  tiltWithPseudo(voltage_gen, R[0], tilt_angle, yycons, voltage_gen.rf_electrodes, debug=False)
cons = [('xx', xxt), ('yy', yycons), ('zz', zzt), ('xz', xzt), ('yz', 0), ('xy', 0)]
# cons = [('yy', yycons), ('yz', 0), ('xy', 0)]

t = time.time()
nom_voltages = voltage_gen.findControlVoltages(
    R,
    cons=[cons]*1,
    tol=1e-13,
    fixed_voltages=constrained_voltages,
    bnds=(-9.9, 9.9),
    epss=1e-6
)

shift_voltages = []
for ind, direction in enumerate(['x', 'y', 'z']):
    # Add a small field in the particular direction by adding a linear constraint
    R = [trap_pos]
    if direction == 'y':
        newcons = cons + [(direction, -1e-4)] #Sign convention is done to be consistent with a positive displacement
    else: # Try fitting with a larger shift and dividing down
        newcons = cons + [(direction, -1e-3)] #Sign convention is done to be consistent with a positive displacement

    # get the voltages needed for a shift by subtracting the nominal voltages
    shift_voltage_set = voltage_gen.findControlVoltages(
        R,
        cons=[newcons],
        tol=1e-13,
        fixed_voltages=constrained_voltages,
        bnds=(-9.9, 9.9),
        epss=-1e-6
    )
    shift_voltage_set -= nom_voltages
    
    if not direction == 'y':
        shift_voltage_set /= 10.0
    shift_voltages.append(shift_voltage_set)

# Save to file

nom_voltages_file_path = os.path.join(output_path, "NominalVolts_{}.npy".format(filename_spec))
nom_voltages_dict = {num_to_electrode_name(num): v for [num, v] in voltage_gen.ungroup_configuration(nom_voltages)}
np.save(nom_voltages_file_path, nom_voltages_dict)

el_config = []
for [num, voltage] in voltage_gen.ungroup_configuration(nom_voltages):
    el_config.append([[num], voltage])
el_config.sort()
print(el_config)

for axis, shift_v in zip(['X', 'Y', 'Z'], shift_voltages):
    shift_voltages_file_path = os.path.join(output_path, "{}DeltaVolts_{}.npy".format(axis, filename_spec))
    shift_voltages_dict = {num_to_electrode_name(num): v for [num, v] in voltage_gen.ungroup_configuration(shift_v)}
    np.save(shift_voltages_file_path, shift_voltages_dict)
    
# np.save(os.path.join(output_path, "Shims_{}.npy".format(filename_spec)), np.array([0., 0., 0.]))

[(3, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (12, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (28, 8), (31, 8), (17, -5.597347664697586), (16, 5.2902534215513), (15, -5.617763846316864), (14, 4.87897587086919), (2, -1.063478070573189), (29, 0.11024014905738937), (1, 0.48728710866056835), (30, -0.09088604221675678), (0, 0.6757519347558454), (25, -3.1345600112211294), (5, 0.7957012684322433), (26, 0.1796437443686206), (4, 0.7866260497899049), (27, -3.0611287716426503)]
Target, Realized Coeffs:
 [[ 0.00000000e+00 -6.22867043e-07]
 [ 0.00000000e+00  1.35674902e-04]
 [ 0.00000000e+00 -3.01547072e-04]
 [-3.13875790e-05 -3.09320021e-05]
 [ 0.00000000e+00 -8.19579301e-12]
 [ 1.73055894e-05  1.72795351e-05]
 [ 8.18154472e-06  7.65066631e-06]
 [ 0.00000000e+00  2.37903596e-06]
 [ 2.32060342e-05  2.40985792e-05]]
Final cost value:  0.0016546326335283548
Number of iterations:  12 

Target, Realized Coeffs:
 [[-1.00000000e-03 -6.22867043e-07]
 [ 0.00000000e+00  

**Plots achieved potential**

In [27]:
el_config.append([[voltage_gen.rf_electrodes[0]], 1]) ## adds rf electrode

R = [[0.0, 0, 50.0]]
y_span_um = 780
n_y_points = 780
y_points = np.linspace(-y_span_um, y_span_um, n_y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R

y_pot = voltage_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in el_config])
plt.plot(y_points, y_pot)

# peaks = sp.signal.find_peaks(-y_pot)[0]
# plt.plot([y_points[p]*1e6 for p in peaks], [y_pot[p] for p in peaks], 'rs')

# ion_min = y_points[peaks[-1]]*1e6
# print(get_freq_from_alpha(np.gradient(np.gradient(y_pot))[peaks[-1]], m_=Ca)/2)